In [1]:
import numpy as np
import pandas as pd
import random


In [11]:
VM_catalogue = pd.read_excel("VM_Catalogue.xlsx",skiprows=1)
workload = pd.read_excel("workload.xlsx")
CPU_VM = VM_catalogue["CPU"].values
RAM_VM = VM_catalogue["RAM"].values
COUT_VM = VM_catalogue["Cost"].values
DISPO_VM = VM_catalogue["Quantity"].values
print(CPU_VM)
# Extraction des paramètres composants
ReqCPU = np.array([1, 1, 1, 1])
ReqRAM = np.array([0.25, 0.25, 0.25, 0.25])
mu = np.array([1416, 696, 1005, 1259])

# λ_i = max de la trace
lamb = workload.max()["input_rate"]
print(lamb)

[  1   2   2   2   4   4   4   8   8   8  16  16  16  40  80  96 160]
5751


In [13]:
NB_COMP = len(ReqCPU)
NB_VM = len(CPU_VM)
print(NB_VM)
def replicas_minimales():
    """Calcule r_i minimal pour la stabilité."""
    return np.ceil(lamb / mu).astype(int)

17


In [14]:
def placement_glouton(r, x):
    """
    Construit y_ij si possible.
    Retourne y ou None si impossible.
    """
    y = np.zeros((NB_COMP, NB_VM), dtype=int)
    ordre_vm = np.argsort(COUT_VM)  # VM les moins chères d'abord

    for i in range(NB_COMP):
        reste = r[i]
        for j in ordre_vm:
            cpu_util = np.dot(y[:, j], ReqCPU)
            ram_util = np.dot(y[:, j], ReqRAM)

            max_cpu = (x[j]*CPU_VM[j] - cpu_util) // ReqCPU[i]
            max_ram = (x[j]*RAM_VM[j] - ram_util) // ReqRAM[i]
            max_possible = int(min(max_cpu, max_ram, reste))

            if max_possible > 0:
                y[i, j] += max_possible
                reste -= max_possible

            if reste == 0:
                break

        if reste > 0:
            return None

    return y


In [15]:
def cout_total(x):
    return np.sum(x * COUT_VM)

def faisable(r, x):
    return placement_glouton(r, x) is not None


In [16]:
def reparer(r, x):
    """Corrige la solution pour garantir faisabilité."""
    
    # Stabilité
    r = np.maximum(r, replicas_minimales())

    # Disponibilité
    x = np.minimum(x, DISPO_VM)

    # Ajouter des VMs si placement impossible
    while placement_glouton(r, x) is None:
        j_best = np.argmin(COUT_VM / CPU_VM)
        if x[j_best] < DISPO_VM[j_best]:
            x[j_best] += 1
        else:
            return None, None

    return r, x


In [17]:
def solution_initiale():
    r = replicas_minimales()
    cpu_total = np.sum(r * ReqCPU)

    j_best = np.argmin(COUT_VM / CPU_VM)
    x = np.zeros(NB_VM, dtype=int)
    x[j_best] = int(np.ceil(cpu_total / CPU_VM[j_best]))

    return reparer(r, x)


In [18]:
def voisin(r, x):
    r2, x2 = r.copy(), x.copy()

    if random.random() < 0.5:
        # Modifier un réplica
        i = random.randint(0, NB_COMP-1)
        r2[i] = max(1, r2[i] + random.choice([-1, 1]))
    else:
        # Modifier un nombre de VM
        j = random.randint(0, NB_VM-1)
        x2[j] = max(0, x2[j] + random.choice([-1, 1]))

    return r2, x2


In [19]:
def hill_climbing(nb_iter=2000):

    r, x = solution_initiale()
    meilleur_cout = cout_total(x)

    for _ in range(nb_iter):
        r2, x2 = voisin(r, x)
        r2, x2 = reparer(r2, x2)

        if r2 is None:
            continue

        nouveau_cout = cout_total(x2)

        if nouveau_cout < meilleur_cout:
            r, x = r2, x2
            meilleur_cout = nouveau_cout

    y = placement_glouton(r, x)
    return r, x, y, meilleur_cout


In [20]:
r_opt, x_opt, y_opt, c_opt = hill_climbing()

print("Coût optimal :", c_opt)
print("Répliques optimales :", r_opt)
print("VM optimales :", x_opt)
print("Placement y_ij :\n", y_opt)


Coût optimal : 1.7648
Répliques optimales : [5 9 6 5]
VM optimales : [0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
Placement y_ij :
 [[0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0]]
